In [2]:
import pandas as pd 
import json
from pandas.io.json import json_normalize 

In [ ]:
##########Carga con Pandas###########

In [ ]:
import time
#exportación y lectura con pandas
inicio = time.time()

pd_Pandas = pd.read_csv('../../../Escritorio/df_data.csv',
                        names='uno')
fin = time.time()

calcultedPD = fin-inicio
print("Cálculo de extracción y lectura con Pandas = {}".format(calcultedPD))

In [ ]:
########Transformación con Pandas################

In [ ]:
#convertir en lista y despues en json
import time
startSecuenciaCPU = time.time()
data = list(pd_Pandas['u'])
da = []
for i in range(0,len(data)):
    d = data[i].split('.gz:')
    da.append(json.loads((d[1])))
    
#DataFrame PANDAS
new_data = pd.DataFrame(da)

#solo conservo los datos de la columna username
#mayor a 0
snNull = new_data[new_data.username.map(len) > 0]

#eliminar username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#eliminar filas con los datos perdidos
snCampus.dropna(axis=0, how='any')

#Eliminar columnas innecesarias no tiene relevancia
snCampus.drop(['accept_language'],axis=1, inplace=True)
snCampus.drop(['name'], axis=1, inplace=True)
snCampus.drop(['agent'], axis=1, inplace=True)
snCampus.drop(['page'], axis=1, inplace=True)
snCampus.drop(['session'], axis=1, inplace=True)
snCampus.drop(['event'], axis=1, inplace=True)
snCampus.drop(['event_type'], axis=1, inplace=True)


#eliminar filas con datos perdidos
snCampus.dropna(axis=0, how='any')



dataLimpia = snCampus

#Dividir toda la columna context
contextData = dataLimpia['context'].values.tolist()
new_contextData = pd.DataFrame(contextData, columns= ['course_user_tags',
                                                      'course_id', 'path',
                                                      'org_id', 'user_id'])
generalData = pd.merge(dataLimpia.reset_index(),
                       new_contextData.reset_index(),
                       left_index=True, right_index=True)
generalData = generalData.drop(['context','index_y', 'course_user_tags',
                               'course_id','path'], axis=1)

#eliminar filas con datos perdidos
generalData.dropna(axis=0, how='any')

#soltar duplicados
snCampus.sort_values("username") 
snCampus.drop_duplicates(subset='username',
                         keep=False, inplace=False)


#dividir columna time
df_picru = pd.DataFrame(generalData['time'])
datenew = df_picru.apply(lambda x: pd.to_datetime(x,errors = 'coerce', format = '%Y-%m-%d'))

datenew['Day'] = datenew['time'].dt.day
datenew['Month'] = datenew['time'].dt.month
datenew['Year'] = datenew['time'].dt.year
datenew['Hour'] = datenew['time'].dt.time

datenew = datenew.drop(['time'], axis=1)
datenew = datenew.drop(['Hour'], axis=1)

#Unir todo el dataframe time al general
generalData = pd.merge(generalData.reset_index(),
                       datenew.reset_index(),
                       left_index=True, right_index=True)
generalData = generalData.drop(['time', 'index_x', 'index_y'], axis=1)

#eliminar filas con datos perdidos
generalData.dropna(axis=0, how='any')

#resetear index
generalData = generalData.reset_index()
generalData = generalData.drop(['index'], axis=1)

endSecuenciaCPU = time.time()
cpuPandas = endSecuenciaCPU - startSecuenciaCPU

print('Cálculo de transformación cpu-Pandas:', cpuPandas)

In [ ]:
#Información de toda la data con Pandas
print('Nomnres de las columnas:',generalData.columns)
print('Dimension del DataFrame:',generalData.shape)
print('Tipos de datos de cada columna', generalData.dtypes)
print('Información DATA general con Pandas:')
generalData.head(2)

In [ ]:
######Carga con Pandas############

In [ ]:
#Guardar la data entendible en un nuevo csv
import time
start = time.time()

da_data = 'data_mejoraPandas.csv'
generalData.to_csv(da_data)

end = time.time()
calculated = end - start

print("Cálculo de carga cpu-Pandas = {}".format(calculated))

In [ ]:
nueva_data = pd.read_csv('data_mejoraPandas.csv')#los datos de las 100000
nueva_data = nueva_data.drop(['Unnamed: 0'],axis=1)

print('Tipos de datos de cada columna', nueva_data.dtypes)
print('Información Data')
nueva_data.head(2)

In [ ]:
#####Proceso Machine learning########

In [24]:
import time
from sklearn.model_selection import train_test_split

inicio_Split = time.time()

#Fase1 de Dividir dataset con sklearn
#Asignar dataset general a nuevas variables
pd_dataX = nueva_data
pd_dataY = nueva_data

#Dividir datasets en train y test
#datatrain 80% y datatest 20%
X_train, X_test, y_train, y_test = train_test_split(pd_dataX, pd_dataY, train_size=0.8)

#información
print(f'Dataset Original: {pd_dataX.shape[0], pd_dataY.shape[0]} elementos')
print(f'Data X_train: {X_train.shape[0]} elementos')
print(f'Data X_test: {X_test.shape[0]} elementos')
print(f'Data y_train: {y_train.shape[0]} elementos')
print(f'Data y_test: {y_test.shape[0]} elementos')
print('')

fin_Split = time.time()
calculo_Split_ML = fin_Split - inicio_Split

print("Cálculo fase dividir dataset con sklearn cpu-sklearn = {}".format(calculo_Split_ML))

Dataset Original: (196867, 196867) elementos
Data X_train: 157493 elementos
Data X_test: 39374 elementos
Data y_train: 157493 elementos
Data y_test: 39374 elementos

Cálculo fase dividir dataset con sklearn cpu-sklearn = 0.05218243598937988


In [ ]:
import time
from sklearn import preprocessing

inicio_Pre = time.time()

#Fase Preprocesamiento con sklearn
le = preprocessing.LabelEncoder()

#seleccionar columnas potenciales para work train de xTrain
value_usernameX = le.fit_transform(X_train.username)
value_EventSourceX = le.fit_transform(X_train.event_source)
value_DayX = le.fit_transform(X_train.Day)

#seleccionar columnas potenciales para work train de xTest
value_usernameXT = le.fit_transform(X_test.username)
value_EventSourceXT = le.fit_transform(X_test.event_source)
value_DayXT = le.fit_transform(X_test.Day)

#seleccionar columnas potenciales para work train de yTrain
value_usernameY = le.fit_transform(y_train.username)
value_EventSourceY = le.fit_transform(y_train.event_source)
value_DayY = le.fit_transform(y_train.Day)

#seleccionar columnas potenciales para work train de yTest
value_usernameYT = le.fit_transform(y_test.username)
value_EventSourceYT = le.fit_transform(y_test.event_source)
value_DayYT = le.fit_transform(y_test.Day)

fin_Pre = time.time()
calculo_Pre_ML = fin_Pre - inicio_Pre

print("Cálculo fase preprocesamiento con sklearn = {}".format(calculo_Pre_ML))

In [ ]:
import time
import numpy as np
from sklearn.linear_model import LinearRegression

#Fase Uso de Algoritmo ML con sklearn :Algoritmo LinearRegression

inicio_ML = time.time()

#Creación de un nuevo dataframe Train
dataTrain = pd.DataFrame()
dataTrain['username'] = np.array(value_usernameX, dtype=np.float32)
dataTrain['day'] = np.array(value_DayX, dtype=np.float32)

#Creación de un nuevo dataframe Test
new_value = np.array(value_usernameY, dtype=np.float32)
dataY = pd.Series(new_value)

lr_reg = LinearRegression(fit_intercept= True, normalize = False)
reg = lr_reg.fit(dataTrain, dataY)

fin_ML = time.time()
calculo_Algorit_ML = fin_ML - inicio_ML

print("Algoritmo Regresión Lineal")
print("Coeficientes:")
print(reg.coef_)
print("Intercepción:")
print(reg.intercept_)
print('Predicción del modelo:')
print(lr_reg.predict(dataTrain))
print('')

print("Cálculo fase de entrenamiento algoritmo Machine Learning con sklearn = {}".format(calculo_Algorit_ML))

In [ ]:
#Algoritmo Regresión Logística de sklearn
import time
import numpy as np
from sklearn.linear_model import LogisticRegression

inicio_RL = time.time()

#Creación de un nuevo dataframe Train 
dataTrain = pd.DataFrame()
dataTrain['username'] = np.array(value_usernameX, dtype=np.float32)
dataTrain['day'] = np.asarray(value_DayX, dtype=np.float32)

#Creación de un nuevo dataframe Test
new_value = np.array(value_usernameY, dtype=np.float32)
dataTest = pd.Series(new_value)

reg = LogisticRegression(fit_intercept = True,
                         max_iter = 100)
reg.fit(dataTrain, dataTest)

fin_RL = time.time()
calculo_RL = fin_RL - inicio_RL

print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
print('Predicción del modelo:')
print(reg.predict(dataTrain))
print('')

print("Cálculo del algoritmo Regresión Logistica con sklearn = {}".format(calculo_RL))

In [ ]:
#######Visualización########

In [ ]:
import matplotlib.pyplot as plt

inicio_grafico = time.time()

dataTrain.plot(kind='bar', x='day',y='username', title='Gráfico de Barras',  width=0.5)

fin_grafico = time.time()

calculo_grafico = fin_grafico - inicio_grafico
print("Cálculo del proceso de Visualización con matplotlib cpu-matplotlib = {}".format(calculo_grafico))

plt.show()
plt.close('all')

In [ ]:
#crear gráfico de líneas estadíssticas
grafico_lin = dataTrain.plot(kind='line', x='day',y='username', title='Gráfico de Lineas Estadísticas',  width=0.5)

plt.show()
plt.close('all')

In [ ]:
#Detalles del proceso con otras herramientas

In [ ]:
#Algoritmo Random Forest de sklearn
"""import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

inicioRF = time.time()

dataTrainUSER = np.array(value_usernameX, dtype=np.float32)
dataTestUSER = np.array(value_usernameY, dtype=np.int32)

dataTrainUSER, dataTestUSER = make_classification(n_samples=10, n_features=4,
                                                 n_informative=2, n_redundant=0,
                                                 random_state=0)

sk_modelRDF = RandomForestClassifier(max_features=1.0,
                             min_samples_split = 2,
                             n_estimators=2)

sk_modelRDF.fit(dataTrainUSER, dataTestUSER)
sk_predict = sk_modelRDF.predict(dataTrainUSER)

finRF = time.time()
calculoRF = finRF - inicioRF

print('Predict labels:', sk_predict)
print('')

print("Cálculo fase de entrenamiento algoritmo Random Forest con sklearn = {}".format(calculoRF))
"""